# CARIA: Seminal Research (Structural Dynamics & Smart Strategy)

## Abstract
We posit that **High Synchronization** is not inherently bad; it is simply **Fragile**.
- **High Sync + Positive Trend** = Asset Bubble (Profitable but Fragile).
- **High Sync + Negative Trend** = Market Crash (The bubble pops).

We implement a **"Smart Exit" Strategy**:
**Rule**: EXIT only if `(Synchronization > Threshold)` AND `(Trend < 0)`.

In [ ]:
# === SETUP ===
!pip install yfinance PyWavelets scikit-learn numpy pandas scipy matplotlib -q
import pandas as pd
import numpy as np
import yfinance as yf
from scipy import stats, signal
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# === ROBUST DATA LOADER ===
def fetch_asset_data(ticker, start_date='2005-01-01'):
    print(f'Fetching data for {ticker}...')
    try:
        df = yf.download(ticker, start=start_date, progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df = df.xs('Close', axis=1, level=0)
        if isinstance(df, pd.DataFrame):
            df = df.iloc[:, 0] # Force Series
        return df.dropna()
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        return pd.Series()

sp500 = fetch_asset_data('^GSPC')
btc = fetch_asset_data('BTC-USD', start_date='2014-01-01')
tlt = fetch_asset_data('TLT')

datasets = {'SP500': sp500, 'BTC': btc, 'TLT': tlt}

In [ ]:
SCALES = {'Fast': 5, 'Medium': 25, 'Slow': 100, 'Macro': 250}

def analyze_dynamics(prices):
    returns = prices.pct_change().dropna()
    phases = {}
    for label, window in SCALES.items():
        # Bandpass Proxy
        val = returns.rolling(window).mean() - returns.rolling(window*2).mean()
        val = val.dropna()
        if len(val) == 0: continue
        
        # Hilbert Phase
        sig_vals = val.values.flatten()
        analytic = signal.hilbert(sig_vals - np.mean(sig_vals))
        phases[label] = pd.Series(np.angle(analytic), index=val.index)
        
    phase_df = pd.DataFrame(phases).dropna()
    if phase_df.empty: return pd.DataFrame()
    
    # Synchronization (r)
    complex_vectors = np.exp(1j * phase_df.values)
    r = np.abs(complex_vectors.mean(axis=1))
    
    return pd.Series(r, index=phase_df.index, name='Sync')

In [ ]:
def backtest_strategies(name, prices):
    print(f"\nRunning Strategy on {name}...")
    dynamics = analyze_dynamics(prices)
    if dynamics.empty: 
        print("No dynamics data")
        return
    
    # Align Data
    df = pd.DataFrame({'Price': prices, 'Sync': dynamics}).dropna()
    df['Returns'] = df['Price'].pct_change()
    
    # === SIGNALS ===
    # 1. Structural Threshold (Fragility)
    threshold = df['Sync'].rolling(252).quantile(0.8)
    is_fragile = df['Sync'] > threshold
    
    # 2. Trend Filter (Momentum)
    # Trend < 0 means price is dropping (20-day ROC)
    trend_20d = df['Price'].pct_change(20)
    is_downtrend = trend_20d < 0
    
    # === STRATEGIES ===
    # A. Naive CARIA: Sell if Fragile (regardless of trend)
    sig_naive = is_fragile.shift(1)
    ret_naive = np.where(sig_naive, 0, df['Returns'])
    
    # B. Smart CARIA: Sell if Fragile AND Downtrend
    sig_smart = (is_fragile & is_downtrend).shift(1)
    ret_smart = np.where(sig_smart, 0, df['Returns'])
    
    # === METRICS ===
    cum_hold = (1 + df['Returns']).cumprod()
    cum_naive = (1 + pd.Series(ret_naive, index=df.index).fillna(0)).cumprod()
    cum_smart = (1 + pd.Series(ret_smart, index=df.index).fillna(0)).cumprod()
    
    sr_hold = df['Returns'].mean() / df['Returns'].std() * np.sqrt(252)
    sr_smart = pd.Series(ret_smart).mean() / pd.Series(ret_smart).std() * np.sqrt(252)
    
    print(f"   Buy & Hold Sharpe: {sr_hold:.2f} | Return: {(cum_hold.iloc[-1]-1)*100:.0f}%")
    print(f"   Smart CARIA Sharpe: {sr_smart:.2f} | Return: {(cum_smart.iloc[-1]-1)*100:.0f}%")
    
    plt.figure(figsize=(10,6))
    plt.plot(cum_hold, label='Buy & Hold', color='gray', alpha=0.5)
    plt.plot(cum_naive, label='Naive Exit (Sync Only)', color='orange', linestyle='--')
    plt.plot(cum_smart, label='Smart Exit (Sync + Trend)', color='cyan', linewidth=2)
    plt.title(f'{name}: "Bubble Rider" Strategy')
    plt.legend()
    plt.yscale('log')
    plt.show()

for name, data in datasets.items():
    backtest_strategies(name, data)